# COMP9517 Group Project

### Model 4 : YOLOv11-nano Segmentation

State-of-the-Art Benchmark

In order to train this state-of-the-art pretrained model as a benchmark for our turtles dataset, we will need to convert our dataset (currently in COCO format) into YOLO format. There are a lot of pre-existing methods which already cover this as YOLO is trained on the COCO dataset, so this is quite straightforward to process. As Ultralytics (the creators of YOLO) already have a tool that does this for us, we will use their tool for this model. Link to this tool can be found here https://github.com/ultralytics/JSON2YOLO

First we will migrate all the images from the subfolders into a single folder

In [ ]:
# Run this to get all images from turtles-data and save them all in a single folder. Then we will partition
# the images into training and testing sets.

import os
import shutil

# Source and destination directories
source_dir = "./turtles-data/data/images/"
destination_dir = "./yolo-dataset/images/"

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

image_extensions = (".jpg", ".jpeg", ".png")

# Traverse through all directories and subdirectories in the source directory
for root, dirs, files in os.walk(source_dir):
    for file in files:
        if file.lower().endswith(image_extensions):
            source_file_path = os.path.join(root, file)
            destination_file_path = os.path.join(destination_dir, file)
            shutil.copy2(source_file_path, destination_file_path)
            print(f"Copied {source_file_path} to {destination_file_path}")

Next, we will check and ensure the lengths of all images matches with labels (no missing labels or images)

In [2]:
# Check how many images are in folder
print(f"Number of images in folder: {len(os.listdir(destination_dir))}")

# Check how many labels are in the labels folder
print(f"Number of labels in folder: {len(os.listdir('./yolo-dataset/labels/'))}")

Number of images in folder: 8709
Number of labels in folder: 8709


Next we will perform the YOLO dataset split in a 80-20 configuration. The file structure looks like this:

```
.
├── seaturtle-seg.YAML
├── images
│   ├── train
│   └── test
└── labels
    ├── train               
    └── test
```

In [10]:
import os
import shutil
import random

# Paths for images and labels
images_dir = "./yolo-dataset/images/"
labels_dir = "./yolo-dataset/labels/"

# Paths for train/test split
train_images_dir = "./yolo-dataset/images/train"
train_labels_dir = "./yolo-dataset/labels/train"
test_images_dir = "./yolo-dataset/images/test"
test_labels_dir = "./yolo-dataset/labels/test"

# Create train/test directories if they don't exist
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(test_images_dir, exist_ok=True)
os.makedirs(test_labels_dir, exist_ok=True)

# Filter only image files (adjust extensions if needed)
image_extensions = (".jpg", ".jpeg", ".png")
image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(image_extensions)]

# Shuffle and split
split_index = int(0.8 * len(image_files))
train_files = image_files[:split_index]
test_files = image_files[split_index:]


# Function to move files
def move_files(
    file_list, src_images_dir, src_labels_dir, dest_images_dir, dest_labels_dir
):
    for image_file in file_list:
        label_file = (
            os.path.splitext(image_file)[0] + ".txt"
        )  # Get corresponding label filename

        # Check if both image and label exist
        src_image_path = os.path.join(src_images_dir, image_file)
        src_label_path = os.path.join(src_labels_dir, label_file)

        if os.path.exists(src_image_path) and os.path.exists(src_label_path):
            # Move image and label to destination
            shutil.move(src_image_path, os.path.join(dest_images_dir, image_file))
            shutil.move(src_label_path, os.path.join(dest_labels_dir, label_file))
            print(
                f"Moved {image_file} and {label_file} to {dest_images_dir} and {dest_labels_dir}"
            )


# Move train and test files
move_files(train_files, images_dir, labels_dir, train_images_dir, train_labels_dir)
move_files(test_files, images_dir, labels_dir, test_images_dir, test_labels_dir)

print("[COMPLETE]: Dataset split into train and test sets with an 80-20 ratio.")

# Check how many in each folder
print(f"Number of images in train folder: {len(os.listdir(train_images_dir))}")
print(f"Number of labels in train folder: {len(os.listdir(train_labels_dir))}")
print(f"Number of images in test folder: {len(os.listdir(test_images_dir))}")
print(f"Number of labels in test folder: {len(os.listdir(test_labels_dir))}")

Moved AabfOCfNfy.JPG and AabfOCfNfy.txt to ./yolo-dataset/images/train and ./yolo-dataset/labels/train
Moved aAfeJIMWyq.JPG and aAfeJIMWyq.txt to ./yolo-dataset/images/train and ./yolo-dataset/labels/train
Moved AaiPbPQCZB.JPG and AaiPbPQCZB.txt to ./yolo-dataset/images/train and ./yolo-dataset/labels/train
Moved aAIPugFRGe.JPG and aAIPugFRGe.txt to ./yolo-dataset/images/train and ./yolo-dataset/labels/train
Moved aAJnyjegFo.JPG and aAJnyjegFo.txt to ./yolo-dataset/images/train and ./yolo-dataset/labels/train
Moved aAlzJMAsWB.JPG and aAlzJMAsWB.txt to ./yolo-dataset/images/train and ./yolo-dataset/labels/train
Moved AaNTfozqoD.JPG and AaNTfozqoD.txt to ./yolo-dataset/images/train and ./yolo-dataset/labels/train
Moved AAObEqwWdy.JPG and AAObEqwWdy.txt to ./yolo-dataset/images/train and ./yolo-dataset/labels/train
Moved aAOhgQfuME.JPG and aAOhgQfuME.txt to ./yolo-dataset/images/train and ./yolo-dataset/labels/train
Moved aatGuMvfJn.JPG and aatGuMvfJn.txt to ./yolo-dataset/images/train an

In [12]:
# Check how many items are in folder
print(f"Number of images in folder: {len(os.listdir(destination_dir))}")

# Check how many items are in the labels folder
print(f"Number of labels in folder: {len(os.listdir('./yolo-dataset/labels/'))}")

Number of images in folder: 2
Number of labels in folder: 2


In [ ]:
from ultralytics import YOLO
import torch

# Try to set gpu
# print(torch.cuda.is_available())
# torch.cuda.set_device(0)

# Load a model
model = YOLO("yolov8n-seg.pt")
print(model.device)

# Train the model
results = model.train(
    data="yolo-dataset/seaturtle-seg.yaml", 
    epochs=5, 
    imgsz=512)

# Validate the model
metrics = model.val()  